<a href="https://colab.research.google.com/github/lagarcian/ProyectodeGradoLuisGarcia/blob/main/Campos_Aleatorios_Gaussianos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

In [ ]:
np.set_printoptions(precision=4)  # Mostrar con solo cuatro decimales en notación científica
np.set_printoptions(suppress=True) # Mostrar como cero los valores muy pequeños en notación científica

El siguiente código define una matriz de covarianza $ \Sigma$ para un conjunto de nodos en el dominio $ [0,1] \times [0,1] $. El número de nodos está dado por la variable $ M $, y se utiliza una función de covarianza exponencial para calcular los elementos de $ \Sigma $. Los parámetros de la función de covarianza son la varianza $ \sigma^2 $ y la longitud de correlación $ l $. Los nodos están organizados en una disposición secuencial en el dominio $ [0,1] \times [0,1] $, y las coordenadas de los nodos se almacenan en la lista llamada "nodos".


In [ ]:
M =2 # Número de puntos en [0,1]x[0,1]

#Se usa una función de covarianza exponencial
# C(t1, t2) = sigma2 * exp(- norm(t1 - t2)/l)
#de parámetros:
sigma2 = 1    # Varianza (constante)
l = 1         # Longitud de correlación

# Los nodos t_n del dominio D = [0,1]x[0,1] se supondrán ordenados en una disposición secuencial:
#[(0,0), (0,1/M),..., (0,(M-1)/M), (1,0),...,(1,(M-1)/M),...,((M-1)/M, 0),...,((M-1)/M,(M-1)/M)]

#Secuencia de coordenadas de nodos en el mismo orden que aparecerán en
#las filas y columnas de la matriz de covarianza Sigma
nodos = [(i/M,j/M) for i in range(M) for j in range(M)]

In [ ]:
len(nodos)
#print(nodos)

In [ ]:
x,y = zip(*nodos) #Matriz de Covarianza de tamaño (MxM, MxM)
X1, X2 = np.meshgrid(x,x)
Y1, Y2 = np.meshgrid(y,y)
# Utilizaremos una función de covarianza exponencial
Sigma = sigma2 * np.exp(-np.sqrt((X1 - X2)**2 +(Y1 - Y2)**2)/l)
print(Sigma)

In [ ]:
Sigma.shape
print(Sigma)
im = plt.imshow(Sigma, cmap='inferno')
plt.colorbar(im);

In [ ]:
L = np.linalg.cholesky(Sigma) #L es la matriz triangular superior, la descomopocisión Cholesky LL^T=Sigma
#print(L)

In [ ]:
#Generamos ahora el vector normal Z∼N(0,I), el cual, al transformarlo linealmente mediante la matriz L
#nos proporcionará una realización de un campo gaussiano N(0,Σ)
#Otras realizaciones se podrían obtener generando un nuevo vector Z

np.random.seed(1)    # Fijamos una semilla para hacer reproducible el resultado
Z = np.random.randn(len(nodos))
N = np.dot(L, Z)

In [ ]:
print(Z)
print(L)
print(N)

In [ ]:
len(N)

In [ ]:
Z2D = np.zeros((M,M))
for i in range(len(nodos)):
    x, y = divmod(i,M)
    Z2D[x,y] = Z[i]

In [ ]:
print(Z2D)

In [ ]:
N2D = np.zeros((M,M))
for i in range(len(nodos)):
    x, y = divmod(i,M)
    N2D[x,y] = N[i]
    #Finalmente vamos a representar gráficamente el array resultante con los valores del campo.
    #Para ello generaremos un array bidimensional para situar en cada punto de la región cuadrada [0,1] x [0,1] el valor del campo escalar que corresponda

In [ ]:
#print(N2D)

In [ ]:
im = plt.imshow(Z2D, cmap='inferno')
plt.colorbar(im);

In [ ]:
im = plt.imshow(N2D, cmap='inferno')
plt.colorbar(im);

In [ ]:
def GRF(M, sigma2, l):
    '''Genera una realización aleatoria de un campo aleatorio gaussiano
    de vector de medias = 0, en la región [0,1]x[0,1] del plano
    utilizando una función de covarianza exponencial
    de parámetros sigma2, l, es decir:
    C(t1, t2) = sigma2 * exp(- norm(t1 - t2)/l)
    sigma2 es la varianza del campo en cada punto.
    l es la longitud de correlación.
    M es el número de subdivisiones de cada eje.
    La función devuelve un array de dimensiones (M, M) con el valor
    del campo escalar en cada punto
    '''
    # Los nodos t_n del dominio D = [0,1]x[0,1] se supondrán ordenados secuencialmente:
    #[(0,0), (0,1/M),...,(0,(M-1)/M),(1/M,0),...,(1/M,(M-1)/M),...,((M-1)/M, 0),...,((M-1)/M,(M-1)/M)]
    #generaremos una lista con las coordenadas de los nodos en el mismo orden que aparecerán en
    #las filas y columnas de la matriz de covarianza
    nodos = [(i/M,j/M) for i in range(M) for j in range(M)]
    #Generación de la matriz de covarianza Sigma
    x,y = zip(*nodos)
    X1, X2 = np.meshgrid(x,x)
    Y1, Y2 = np.meshgrid(y,y)
    Sigma = sigma2 * np.exp(-np.sqrt((X1 - X2)**2 +(Y1 - Y2)**2)/l)
    #Ahora hacemos la descomposición de Cholesky de Sigma
    L = np.linalg.cholesky(Sigma)
    np.random.seed(1)    #Se fija una semilla para hacer reproducible el resultado
    Z = np.random.randn(len(nodos))    # Vector de M*M variables independientes N(0,1)
    N = np.dot(L, Z) #Se genera un vector aleatorio gaussiano N(0, Sigma) de longitud M*M
    N2D = np.zeros((M,M)) #Se reorganiza el N en dos dimensione.
    for i in range(len(nodos)):
        a, b = divmod(i,M)    #indices en el array N2D
        N2D[a,b] = N[i]
    return N2D

In [ ]:
def WN(M, sigma2, l):
    '''Genera una realización aleatoria de un campo aleatorio gaussiano
    de vector de medias = 0, en la región [0,1]x[0,1] del plano
    utilizando una función de covarianza exponencial
    de parámetros sigma2, l, es decir:
    C(t1, t2) = sigma2 * exp(- norm(t1 - t2)/l)
    sigma2 es la varianza del campo en cada punto.
    l es la longitud de correlación.
    M es el número de subdivisiones de cada eje.
    La función devuelve un array de dimensiones (M, M) con el valor
    del campo escalar en cada punto
    '''
    # Los nodos t_n del dominio D = [0,1]x[0,1] se supondrán ordenados secuencialmente:
    #[(0,0), (0,1/M),...,(0,(M-1)/M),(1/M,0),...,(1/M,(M-1)/M),...,((M-1)/M, 0),...,((M-1)/M,(M-1)/M)]
    #generaremos una lista con las coordenadas de los nodos en el mismo orden que aparecerán en
    #las filas y columnas de la matriz de covarianza
    nodos = [(i/M,j/M) for i in range(M) for j in range(M)]
    #Generación de la matriz de covarianza Sigma
    x,y = zip(*nodos)
    X1, X2 = np.meshgrid(x,x)
    Y1, Y2 = np.meshgrid(y,y)
    Sigma = sigma2 * np.exp(-np.sqrt((X1 - X2)**2 +(Y1 - Y2)**2)/l)
    #Ahora hacemos la descomposición de Cholesky de Sigma
    L = np.linalg.cholesky(Sigma)
    np.random.seed(1)    #Se fija una semilla para hacer reproducible el resultado
    Z = np.random.randn(len(nodos))    # Vector de M*M variables independientes N(0,1)
    #N = np.dot(L, Z) #Se genera un vector aleatorio gaussiano N(0, Sigma) de longitud M*M
    Z2D = np.zeros((M,M)) #Se reorganiza el N en dos dimensione.
    for i in range(len(nodos)):
        a, b = divmod(i,M)    #indices en el array Z2D
        Z2D[a,b] = Z[i]
    return Z2D

In [ ]:
M = 100
positions = np.linspace(0,M,7)
labels = np.linspace(0,1,2)
fig, ax = plt.subplots(2,2, figsize=(15, 12))
im = ax[0,0].imshow(GRF(M, 1, 1).T, origin='lower', cmap='inferno')
ax[0,0].set_title(r'GRF $\sigma^2 = 1$, $\lambda=1$', fontsize=16)
fig.colorbar(im, ax = ax[0,0])

im = ax[0,1].imshow(GRF(M, 1, 0.1).T, origin='lower' , cmap='inferno')
ax[0,1].set_title(r'GRF $\sigma^2 = 1$, $\lambda=0.1$', fontsize=16)
fig.colorbar(im, ax = ax[0,1])

im = ax[1,0].imshow(GRF(M, 9, 1).T, origin='lower', cmap='inferno')
ax[1,0].set_title(r'GRF $\sigma^2 = 9$, $\lambda=1$', fontsize=16)
fig.colorbar(im, ax = ax[1,0])

im = ax[1,1].imshow(GRF(M, 9, 0.1).T, origin='lower', cmap='inferno')
ax[1,1].set_title(r'GRF $\sigma^2 = 9$, $\lambda=0.1$', fontsize=16)
fig.colorbar(im, ax = ax[1,1])

for i in [0,1]:
    for j in [0,1]:
        ax[i,j].set_xticks(positions)
        ax[i,j].set_xticklabels(labels)
        ax[i,j].set_yticks(positions)
        ax[i,j].set_yticklabels(labels)

In [ ]:
M = 50
positions = np.linspace(0,M, 6)
labels = np.linspace(0,1,6)
fig, ax = plt.subplots(2,2, figsize=(15, 12))
im = ax[0,0].imshow(GRF(M, 1, 1).T, origin='lower')
ax[0,0].set_title(r'GRF $\sigma^2 = 1$, $l=1$', fontsize=16)
fig.colorbar(im, ax = ax[0,0])

In [ ]:
fig = plt.figure(figsize=(7,7))
ax1 = fig.add_subplot(111, projection='3d')
N = GRF(M, 1, 1)
ax1.plot_surface(X, Y, N, rstride=1, cstride=1, cmap=cm.coolwarm, linewidth=0.5)
ax1.set_zlim(-5, 5)
ax1.set_title(r'GRF $\sigma^2 = 1$, $l=1$', fontsize=16);

In [ ]:
im2 = plt.imshow(WN(120, 1, 1).T, origin='lower', cmap='inferno')
plt.colorbar(im2)
plt.figure()
im = plt.imshow(GRF(120, 1, 1).T, origin='lower', cmap='inferno')
plt.colorbar(im)
plt.figure()
